In [368]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.optimize import minimize
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

Импортируем датасеты

In [369]:
frame=pd.read_csv('train.csv')
test=pd.read_csv("test.csv")

Приводим все значения NaN и None к 0

In [370]:
frame.replace(np.nan,0,inplace=True)
frame.replace('None',0,inplace=True)
test.replace(np.nan,0,inplace=True)
test.replace('None',0,inplace=True)

В частном парядке исправляем типы данныхв столбцах,которые были object, но хранили числовые значения

In [371]:
frame['GarageYrBlt']=frame['GarageYrBlt'].astype('float')
frame['LotFrontage']=frame['LotFrontage'].astype('float')
frame['MasVnrArea']=frame['MasVnrArea'].astype('float')
test['LotFrontage']=test['LotFrontage'].astype('float')
test['GarageYrBlt']=test['GarageYrBlt'].astype('float')
test['MasVnrArea']=test['MasVnrArea'].astype('float')
test['BsmtFinSF1']=test['BsmtFinSF1'].astype('float')

In [372]:
frame

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,0,Reg,Lvl,AllPub,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,0,Reg,Lvl,AllPub,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,0,IR1,Lvl,AllPub,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,0,IR1,Lvl,AllPub,...,0,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,0,IR1,Lvl,AllPub,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,0,Reg,Lvl,AllPub,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,0,Reg,Lvl,AllPub,...,0,0,MnPrv,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,0,Reg,Lvl,AllPub,...,0,0,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,0,Reg,Lvl,AllPub,...,0,0,0,0,0,4,2010,WD,Normal,142125


создаем True/False списки, которые отделяют столбцы с типом object от остальных

In [373]:
mask=frame.dtypes == object
masttest=test.dtypes == object

инверсируем списки

In [374]:
maskI=mask ==False
testI=masttest == False

создаем списки названий столбцов(первые два с числовыми значениями, оставшиеся с категорийными) 

In [375]:
columnsI=frame.columns[maskI].tolist()
columnsIT=test.columns[testI].tolist()
columns=frame.columns[mask].tolist()
columnsTest=test.columns[masttest].tolist()

Приводим каждый столбец в соответсующий тип

In [376]:
for  i in columnsI:
    frame[i]=frame[i].astype(float)
for  i in columnsIT:
    test[i]=test[i].astype(float)
for i in columns:
    frame[i]=frame[i].astype(str)
for i in columnsTest:
    test[i]=test[i].astype(str)

создаем уникальный список уникальный категорий

In [377]:
types=[]
for j in columns:
    for i in frame[j]:
        if i in types:
            pass
        else:
            types.append(i)

пустой список или нет

In [ ]:
bool(types)

тренеруюем кодировщик признаков

In [379]:
le.fit_transform(types)

array([133, 134,  33,  57, 132, 126,  78,   0, 140,  84,  85,  86,  90,
        24,  89,  80,  13, 114,  88,  55,  48,  50,  56,  79, 106, 148,
        42, 163,  49, 113, 101, 151, 110, 118,  30, 146, 117, 108, 147,
        83,  95,  53, 158,  76, 153,  39, 109,  22,  25, 145,  23, 115,
        60, 130,  15, 135, 138, 136, 129, 137,   3,   9,  52, 160, 159,
         7,   4,   1,   2, 143, 144,   6,   5,  68,  82,  69,  94,  62,
       149,  43, 171,  97, 169,  96, 156, 141,  40, 164,  98, 167,  81,
        29, 170,  38, 127,  16, 154,  28,  17, 152,  87,  34, 168,  41,
        26, 121,  27,  73, 155,  54,  58, 128, 124,  31, 172, 150, 112,
       103,  19,  67,  10, 162, 139,  20,  91,  71,  72,  77, 166, 120,
        63, 173, 107, 142,  65,  64,  66, 102, 161,  99,  92, 100,  93,
        18,  51,  32,  37,  21,   8, 131,  61, 123, 104,  75,  74, 105,
        70, 122, 157, 165, 111,  35,  45,  46,  36,  47,  44, 119, 116,
        11, 125,  12,  14,  59], dtype=int32)

трансформируем таблицы в соответсвии с таблицей кодировки

In [380]:
frame[columns]=frame[columns].apply(lambda col: le.transform(col))
test[columnsTest]=test[columnsTest].apply(lambda col: le.transform(col))

In [381]:
frame

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,133,65.0,8450.0,126,0,140,90,13,...,0.0,0,0,0,0.0,2.0,2008.0,165,116,208500.0
1,2.0,20.0,133,80.0,9600.0,126,0,140,90,13,...,0.0,0,0,0,0.0,5.0,2007.0,165,116,181500.0
2,3.0,60.0,133,68.0,11250.0,126,0,84,90,13,...,0.0,0,0,0,0.0,9.0,2008.0,165,116,223500.0
3,4.0,70.0,133,60.0,9550.0,126,0,84,90,13,...,0.0,0,0,0,0.0,2.0,2006.0,165,11,140000.0
4,5.0,60.0,133,84.0,14260.0,126,0,84,90,13,...,0.0,0,0,0,0.0,12.0,2008.0,165,116,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,133,62.0,7917.0,126,0,140,90,13,...,0.0,0,0,0,0.0,8.0,2007.0,165,116,175000.0
1456,1457.0,20.0,133,85.0,13175.0,126,0,140,90,13,...,0.0,0,104,0,0.0,2.0,2010.0,165,116,210000.0
1457,1458.0,70.0,133,66.0,9042.0,126,0,140,90,13,...,0.0,0,74,149,2500.0,5.0,2010.0,165,116,266500.0
1458,1459.0,20.0,133,68.0,9717.0,126,0,140,90,13,...,0.0,0,0,0,0.0,4.0,2010.0,165,116,142125.0


In [382]:
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461.0,20.0,132,80.0,11622.0,126,0,140,90,13,...,120.0,0.0,0,104,0,0.0,6.0,2010.0,165,116
1,1462.0,20.0,133,81.0,14267.0,126,0,84,90,13,...,0.0,0.0,0,0,70,12500.0,6.0,2010.0,165,116
2,1463.0,60.0,133,74.0,13830.0,126,0,84,90,13,...,0.0,0.0,0,104,0,0.0,3.0,2010.0,165,116
3,1464.0,60.0,133,78.0,9978.0,126,0,84,90,13,...,0.0,0.0,0,0,0,0.0,6.0,2010.0,165,116
4,1465.0,120.0,133,43.0,5005.0,126,0,84,80,13,...,144.0,0.0,0,0,0,0.0,1.0,2010.0,165,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915.0,160.0,134,21.0,1936.0,126,0,140,90,13,...,0.0,0.0,0,0,0,0.0,6.0,2006.0,165,116
1455,2916.0,160.0,134,21.0,1894.0,126,0,140,90,13,...,0.0,0.0,0,0,0,0.0,4.0,2006.0,165,11
1456,2917.0,20.0,133,160.0,20000.0,126,0,140,90,13,...,0.0,0.0,0,0,0,0.0,9.0,2006.0,165,11
1457,2918.0,85.0,133,62.0,10441.0,126,0,140,90,13,...,0.0,0.0,0,104,149,700.0,7.0,2006.0,165,116


делим тренеровочную выборку на тренеровочную и тестовую в соотношении 80/20

In [388]:
train=frame[frame['Id']<=(len(frame)*0.8)]
testFrame=frame[frame['Id']>(len(frame)*0.8)]

разделяем фичи и ответ для тренеровочной выборки

In [501]:
testX=testFrame.drop(['Id','SalePrice'],axis=1)
testY=testFrame['SalePrice']

разделяем фичи и ответы для тестовой выборки

In [502]:
trainX=train.drop(['Id','SalePrice'],axis=1)
trainY=train['SalePrice']

импортируем дерево решений

In [503]:
from sklearn.tree import DecisionTreeRegressor

создаем и тренеруем дерево решений

In [528]:
regTree=DecisionTreeRegressor(max_depth=5,max_features='auto')
regTree.fit(trainX,trainY)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

предсказываем ответы на тестовой выборке

In [529]:
pred=regTree.predict(testX)


считаем среднюю ошибку

In [531]:
sum(abs(np.array(testY)-pred)/len(testY))

28920.50828907898

импортируем библиотеке для отрисовки решающего дерева

In [532]:
import graphviz
from sklearn import tree
from os import system 

создаем список всех фич

In [533]:
labels=testX.columns.tolist()

генерируем отображение дерева

In [534]:
tree.export_graphviz(regTree, out_file="tree3.dot",feature_names=labels)

занимаемся созданием фрейма соответствия кодировки и категорийного признака

In [519]:
real=le.classes_

In [520]:
encode=le.transform(real)


создаем фрейм из полученных данных

In [544]:
var=pd.DataFrame(real,encode)

0    Mitchel
Name: 101, dtype: object

записываем его в файл

In [537]:
var.to_csv('labels.csv')